In [2]:
from EDTpy.ChipElements import *

In [3]:
import numpy as np
import copy

In [4]:
from EDTpy import settings

In [5]:
import EDTpy
from EDTpy import EmptyGeometry
from EDTpy import EmptyPath
from EDTpy.settings import *
import gdspy

In [338]:
# setting cpw parameters
CPW.default_values['S'] = 10
CPW.default_values['W'] = 6
CPW.default_values['layer'] = 0

In [339]:
class Inline_capasitor(EmptyGeometry):
    default_values = {
        "s": CPW.default_values['S'],
        "w": CPW.default_values['W'],
        "N": 3, 
        "l": 100, 
        "a": 3.3,  
        "b": 3.3, 
        "gap": 3.3, 
        "contact_pad_size": 200,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "CPW_C"
        
        w = values['w']
        s = values['s']
        N = values['N']
        a = values['a']
        b = values['b']
        l = values['l']
        gap = values['gap']
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']
        
        self + gdspy.Rectangle([0, -s/2-w],
                               [10, s/2+w], layer = 0)
        self - gdspy.Rectangle([0, -s/2],
                               [10, s/2], layer = 0)
        
        jo = N*(a+b)*2-b
        jo_g = jo+10*s
        offset = 20
        points = [(10, s/2+w), 
                  (offset, jo_g/2), (offset+20, jo_g/2), (offset+20+l, jo_g/2), 
                  (offset+l+20, jo/2), (offset+20, jo/2), (offset, jo/2), (10, s/2)]
        self + gdspy.Polygon(points)
        points = [(10, -s/2-w), 
                  (offset, -jo_g/2), (offset+20, -jo_g/2), (offset+l+20, -jo_g/2), 
                  (offset+l+20, -jo/2), (offset+20, -jo/2), (offset, -jo/2), (10, -s/2)]
        self + gdspy.Polygon(points)
        
        offset += 10
        self + gdspy.Rectangle([offset, jo/2],
                               [offset+l, -jo/2], layer = 0)
        for n in range(N):
            self - gdspy.Rectangle([offset, -jo/2+2*(a+b)*n],
                                   [offset+l-gap, -jo/2+2*(a+b)*n+a], layer = 0)
            self - gdspy.Rectangle([offset+gap, -jo/2+2*(a+b)*n+a+b],
                                   [offset+l, -jo/2+2*(a+b)*n+a+b+a], layer = 0)
        
        z = offset+l+10
        points = [(z, jo_g/2), (z+20, s/2+w), (z+30, s/2+w), 
                  (z+30, s/2), (z+20, s/2), (z, jo/2)]
        self + gdspy.Polygon(points)
        points = [(z, -jo_g/2), (z+20, -s/2-w), (z+30, -s/2-w), 
                  (z+30, -s/2), (z+20, -s/2), (z, -jo/2)]
        self + gdspy.Polygon(points)

        self.add_port([0, 0], 180)
        self.add_port([z+30, 0], 0)

In [340]:
# setting graphen hole
class GraphenHole(EmptyGeometry):
    default_values = {
        "size": 200,
        "layer": 0,
        "contact_S": CPW.default_values['S'],
        "contact_b": CPW.default_values['S'] + 2*CPW.default_values['W'],
        "contact_L": 30,        
    }

    def _drawing(self, values):
        self.name = "GraphenHole"
        
        self + gdspy.Rectangle([-values['size']/2, -values['size']/2],
                               [values['size']/2, values['size']/2],
                               layer = values['layer'])
        
        #Markers
        mark = gdspy.boolean(gdspy.Rectangle([-55, 50.6],[-45, 49.4]), gdspy.Rectangle([-50.6, 55],
                               [-49.4, 45]), 'or', layer = 0)
       
        for i in range(4):
            mark.rotate(np.pi/2)
            self - mark
        
        
        leg_central = gdspy.Rectangle([-values['size']/2, -values['contact_S']/2] ,
                               [-values['size']/2 + values['contact_L'], values['contact_S']/2],
                               layer = values['layer'])
        re_central = gdspy.Rectangle(
                               [-values['size']/2 + values['contact_L']+10, -values['contact_S']/2-5] ,
                               [-values['size']/2 + values['contact_L'], values['contact_S']/2+5],
                               layer = values['layer'])
        
        
        leg_corner_S = gdspy.Rectangle([-values['size']/2*np.sqrt(2), -values['contact_S']/2],
                               [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2), values['contact_S']/2],
                               layer = values['layer'])
        re_corner_S = gdspy.Rectangle(
                           [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2)+10, -values['contact_S']/2-5],
                           [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2), values['contact_S']/2+5],
                           layer = values['layer'])
        
        leg_corner_W = gdspy.Rectangle([-values['size']/2*np.sqrt(2), -values['contact_b']/2],
                               [-values['size']/2*np.sqrt(2) + values['contact_L']*np.sqrt(2), values['contact_b']/2],
                               layer = values['layer'])
        
        leg_corner_S.rotate(45/180*np.pi)
        re_corner_S.rotate(45/180*np.pi)
        leg_corner_W.rotate(45/180*np.pi)
        
#         leg_central.rotate(90/180*np.pi)
#         re_central.rotate(90/180*np.pi)

        
        for i in range(4):
            self - leg_central.rotate(90/180*np.pi*i)
            self - re_central.rotate(90/180*np.pi*i)
            
            self + leg_corner_W.rotate(90/180*np.pi*i)
            self - leg_corner_S.rotate(90/180*np.pi*i)
            self - re_corner_S.rotate(90/180*np.pi*i)
        
        port_central = EDTpy.Port((-values['size']/2, 0), 180)
        port_corner = EDTpy.Port((-values['size']/2, values['size']/2), 180-45)

        for i in range(4):
            self.ports += [copy.deepcopy(port_central)]
            self.ports += [copy.deepcopy(port_corner)]
            port_central.rotate(90, center = (0, 0))
            port_corner.rotate(90, center = (0, 0))
            
    
        #add capasitor inline
        cap = Inline_capasitor()
        cap.merge_with(self.ports[0], 0)
        self+cap
        self.add_port(cap.ports[1].position, 180)
        cap = Inline_capasitor()
        cap.merge_with(self.ports[4], 0)
        self+cap
        self.add_port(cap.ports[1].position, 0)

In [341]:
# setting contact pad
# sketch = EDTpy.EmptySketch()
# test = GraphenHole()
# sketch.add_geometry(test)
# test.show()

In [342]:
# spiral
class Contact_pad_spiral(EmptyGeometry):
    default_values = {
        "a": 7,
        "b": 9,
        "N_sp": 6,
        "contact_pad_size": 150,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Contact_pad_spiral"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']

        self + gdspy.Rectangle([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, (contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)],
                                 [-(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3), -(contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3)], layer = 0)

        
        self - gdspy.Rectangle([-contact_pad_size/2, -contact_pad_size/2],
                                 [contact_pad_size/2, contact_pad_size/2], layer = 0)

        self - gdspy.Rectangle([-a/2, contact_pad_size/2],
                                 [a/2, contact_pad_size/2+b], layer = 0)
        self - gdspy.Rectangle([-a/2, contact_pad_size/2+b],
                                 [contact_pad_size/2+b, contact_pad_size/2+b+a], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(i), contact_pad_size/2+b+a+(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [contact_pad_size/2+b+a+(a+b)*(i), -contact_pad_size/2-b-(a+b)*(i)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-(a+b)*(i), -contact_pad_size/2-b-a-(a+b)*(i)],
                                     [-contact_pad_size/2-b-a-(a+b)*(i), +contact_pad_size/2+b+(a+b)*(i+1)], layer = 0)

            self - gdspy.Rectangle([-contact_pad_size/2-b-a-(a+b)*(i), contact_pad_size/2+b+(a+b)*(i+1)],
                                     [contact_pad_size/2+b+(a+b)*(i+1), contact_pad_size/2+b+a+(a+b)*(i+1)], layer = 0)

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), contact_pad_size/2+b+a+(a+b)*(N_sp)],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp), a/2], layer = 0)  

        self - gdspy.Rectangle([contact_pad_size/2+b+(a+b)*(N_sp), a/2],
                                 [contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, -a/2], layer = 0)  

        self.add_port([contact_pad_size/2+b+a+(a+b)*(N_sp)+b*3, 0], 0)
        self.add_port([0, 0], 180)

In [343]:
class Planar_capasitor(EmptyGeometry):
    default_values = {
        "a": CPW.default_values['S'],
        "b": CPW.default_values['W'],
        "gap": 10,
        "N_sp": 15,
        "width": 120,
        "layer": 0,
        "leg": 60
    }

    def _drawing(self, values):
        self.name = "Planar_capasitor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        layer = values['layer']
        leg = values['leg']

        self + gdspy.Rectangle([0, w],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b, a/2], layer = 0)

        for i in range(N_sp):

            self - gdspy.Rectangle([2*b+(2*a+2*b)*(i), -a/2],
                                   [2*b+(2*a+2*b)*(i+1), a/2], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), w],
                                   [2*b+b+a+(2*a+2*b)*(i), a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+(2*a+2*b)*(i), -w],
                                   [2*b+b+a+(2*a+2*b)*(i), -a/2-gap], layer = 0)
            
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), -w+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -a/2], layer = 0)
            
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+2*b, a/2], layer = 0)
                    
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, w],
                               [(a+b)*2*(N_sp)+b*2+a+b, a/2+gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+b*2+a, -w],
                               [(a+b)*2*(N_sp)+b*2+a+b, -a/2-gap], layer = 0)
        
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)

In [344]:
class Planar_inductor(EmptyGeometry):
    default_values = {
        "a": 7,
        "b": 9,
        "gap": 10,
        "N_sp": 12,
        "width": 200,
        "leg":180,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Planar_inductor"
        
        a = values['a']
        b = values['b']
        N_sp = values['N_sp']
        w = values['width']
        gap = values['gap']
        leg = values['leg']
        layer = values['layer']

        self + gdspy.Rectangle([0, w+b],
                               [(a+b)*2*(N_sp)+4*b+a+b, -w-b], layer = 0)

        self - gdspy.Rectangle([0, -a/2],
                               [2*b+b+a, a/2], layer = 0)
        
        self - gdspy.Rectangle([2*b+b, a/2],
                               [2*b+b+a, w+a/2-gap], layer = 0)

        for i in range(N_sp):
            self - gdspy.Rectangle([2*b+b+a+(2*a+2*b)*(i), w+a/2-gap],
                                   [2*b+b+a+b+(2*a+2*b)*(i), w-gap-a/2], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+(2*a+2*b)*(i), -w-a/2+gap],
                                   [2*b+b+a+b+a+(2*a+2*b)*(i)+b, -w-a/2+gap+a], layer = 0)
            self - gdspy.Rectangle([2*b+b+a+b+a+b+(2*a+2*b)*(i), w-gap+a/2],
                                   [2*b+b+a+b+a+a+b+(2*a+2*b)*(i), -w-a/2+gap], layer = 0)

        self + gdspy.Rectangle([2*b+b+(2*a+2*b)*(N_sp), a/2],
                               [2*b+b+a+(2*a+2*b)*(N_sp), w+a/2-gap], layer = 0)
        self - gdspy.Rectangle([(a+b)*2*(N_sp)+a+b+b*2, -a/2],
                               [(a+b)*2*(N_sp)+a+b+b*4, a/2], layer = 0)  
        #leg for connection
        x_start = (a+b)*2*(N_sp)+a+b+b*4
        self + gdspy.Rectangle([x_start, -CPW.default_values['S']/2-CPW.default_values['W']],
                               [x_start+leg, CPW.default_values['S']/2+CPW.default_values['W']], layer = 0)  
        self - gdspy.Rectangle([x_start, -CPW.default_values['S']/2],
                               [x_start+leg, CPW.default_values['S']/2], layer = 0)  

        self.add_port([x_start+leg, 0], 0)
        self.add_port([0, 0], 180)

In [345]:
class Marker(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        
        self + gdspy.Rectangle([-b/2-b/10, -b/2-b/10],
                               [b/2+b/10, b/2+b/10], layer = 0)
        
        mark = gdspy.boolean(gdspy.Rectangle([-b/2, -a/2],[b/2, a/2]), gdspy.Rectangle([-a/2, -b/2],
                               [a/2, b/2]), 'or', layer = 0)
        self - mark
        self.add_port([0, 0], 0)

In [346]:
class CPW_C(EmptyGeometry):
    default_values = {
        "s": CPW.default_values['S'],
        "w": CPW.default_values['W'],
        "N": 3, 
        "l": 100, 
        "a": 3.3,  
        "b": 3.3, 
        "gap": 3.3, 
        "contact_pad_size": 200,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "CPW_C"
        
        w = values['w']
        s = values['s']
        N = values['N']
        a = values['a']
        b = values['b']
        l = values['l']
        gap = values['gap']
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']
        
        self + gdspy.Rectangle([0, -s/2-w],
                               [10, s/2+w], layer = 0)
        self - gdspy.Rectangle([0, -s/2],
                               [10, s/2], layer = 0)
        
        jo = N*(a+b)*2-b
        jo_g = jo+10*s
        offset = 100
        points = [(10, s/2+w), 
                  (offset, jo_g/2), (offset+l, jo_g/2), 
                  (offset+l, jo/2), (offset, jo/2), (10, s/2)]
        self + gdspy.Polygon(points)
        points = [(10, -s/2-w), 
                  (offset, -jo_g/2), (offset+l, -jo_g/2), 
                  (offset+l, -jo/2), (offset, -jo/2), (10, -s/2)]
        self + gdspy.Polygon(points)
        
        self + gdspy.Rectangle([offset, jo/2],
                               [offset+l, -jo/2], layer = 0)
        for n in range(N):
            self - gdspy.Rectangle([offset, -jo/2+2*(a+b)*n],
                                   [offset+l-gap, -jo/2+2*(a+b)*n+a], layer = 0)
            self - gdspy.Rectangle([offset+gap, -jo/2+2*(a+b)*n+a+b],
                                   [offset+l, -jo/2+2*(a+b)*n+a+b+a], layer = 0)
        
        z = offset+l
        self + gdspy.Rectangle([z+100, -contact_pad_size/2-50],
                               [z+100+contact_pad_size+50, contact_pad_size/2+50], layer = 0)
        self - gdspy.Rectangle([z+100, -contact_pad_size/2],
                       [z+100+contact_pad_size, contact_pad_size/2], layer = 0)
        
        points = [(z, jo_g/2), 
                  (z+100, contact_pad_size/2+50), (z+100, contact_pad_size/2), 
                  (z, jo/2)]
        self + gdspy.Polygon(points)
        
        points = [(z, -jo_g/2), 
                  (z+100, -contact_pad_size/2-50), (z+100, -contact_pad_size/2), 
                  (z, -jo/2)]
        self + gdspy.Polygon(points)

        self.add_port([0, 0], 180)

In [410]:
# setting resonator default path
res_R = 65
res_N = 5
res_a = 300
res_ratio = 1/3.6
res_period = res_R*4

y = lambda x: x*res_ratio + res_a/2
res_path = [[-200, 0]]
for i in range(res_N):
    res_path += [[i*res_period, 0],
                 [i*res_period, y(i*res_period + res_period/4)],
                 [i*res_period + res_period/2, y(i*res_period + res_period/4)],
                 [i*res_period + res_period/2, -y(i*res_period + 3*res_period/4)],
                 [i*res_period + res_period, -y(i*res_period + 3*res_period/4)]
                ]
    
res_path.append([res_N*res_period, 0])  
res_path.append(list(np.array(res_path[-1]) + np.array([100, 0])))

In [411]:
res = CPW(res_path, res_R)
res_r = CPW(res_path, res_R)
print(res.length)
# res.show()

7597.349858427811


In [412]:
res.ports[0].basis[1]

array([-1.8369702e-16, -1.0000000e+00])

In [413]:
# setting sketch basis
sketch = EDTpy.EmptySketch()
sketch + gdspy.Rectangle([-5000/2, -5000/2],
                         [5000/2, 5000/2], layer = 0)

sketch - gdspy.Rectangle([-5000/2 + 50, -5000/2 + 50],
                         [5000/2 - 50, 5000/2 - 50], layer = 0)

sketch.add_port([-5000/2 + 700, 0], 0) #resonator port  -- 0
 
sketch.add_port([-5000/2 + 400, 1000], 0) #DC port  -- 1
sketch.add_port([-1000, +5000/2 - 400], -90) #DC port  --  2

sketch.add_port([-2100, 2100], 0) #Marker port  --  3


sketch.add_port([-2100, 2100], 0) #Marker port  --  4
marker = Marker()
marker.merge_with(sketch.ports[4], 0)
sketch.add_geometry(marker)
sketch.circular_array(marker)

sketch.add_port([-200, 50], 0) #Extra Marker port  --  5
markere = Marker(a=2, b=10)
markere.merge_with(sketch.ports[5], 0)
sketch.add_geometry(markere)
sketch.circular_array(markere)

sketch.add_port([-200, -50], 0) #Extra Marker port  --  6
markere = Marker(a=2, b=10)
markere.merge_with(sketch.ports[6], 0)
sketch.add_geometry(markere)
sketch.circular_array(markere)

graphen_hole = GraphenHole()
sketch.add_geometry(graphen_hole)
# graphen_hole.show()

In [414]:
spiral = Contact_pad_spiral()
spiral.merge_with(sketch.ports[1], 1)
sketch.add_geometry(spiral)
cap = Planar_capasitor(width=250, leg = 0)
cap.merge_with(spiral.ports[0], 0)
sketch.add_geometry(cap)
# res_i = CPW(path_i, res_R)
# res_i.merge_with(cap.ports[1], 0)
# sketch.add_geometry(res_i)

spiral1 = Contact_pad_spiral()
spiral1.merge_with(sketch.ports[2], 1)
sketch.add_geometry(spiral1)
cap1 = Planar_capasitor(width=250, leg = 0)
cap1.merge_with(spiral1.ports[0], 0)
sketch.add_geometry(cap1)
# res_i = CPW(path_i, res_R)
# res_i.merge_with(cap1.ports[1], 0)
# sketch.add_geometry(res_i)
# sketch.show()

In [415]:
cap_i = Planar_capasitor(width=150, N_sp = 22, leg = 100)
cap_i.merge_with(graphen_hole.ports[1], 0)
sketch.add_geometry(cap_i)
ind = Planar_inductor(width=200, leg = 20)
ind.merge_with(cap_i.ports[1], 1)
sketch.add_geometry(ind)

#extra lines for gate/DC lines
cap_o = Planar_capasitor(width=200, N_sp = 15, leg = 350)
cap_o.merge_with(graphen_hole.ports[2], 0)
sketch.add_geometry(cap_o)
ind_o = Planar_inductor(width=200, leg = 20)
ind_o.merge_with(cap_o.ports[1], 1)
sketch.add_geometry(ind_o)
cap_o1 = Planar_capasitor(width=200, N_sp = 12, leg = 0)
cap_o1.merge_with(ind_o.ports[0], 0)
sketch.add_geometry(cap_o1)
spiral_o1 = Contact_pad_spiral()
spiral_o1.merge_with(cap_o1.ports[1], 0)
sketch.add_geometry(spiral_o1)

cap_oo = Planar_capasitor(width=200,  N_sp = 15, leg = 350)
cap_oo.merge_with(graphen_hole.ports[6], 0)
sketch.add_geometry(cap_oo)
ind_oo = Planar_inductor(width=200, leg = 20)
ind_oo.merge_with(cap_oo.ports[1], 1)
sketch.add_geometry(ind_oo)
cap_o1o = Planar_capasitor(width=200, N_sp = 12, leg = 0)
cap_o1o.merge_with(ind_oo.ports[0], 0)
sketch.add_geometry(cap_o1o)
spiral_o1o = Contact_pad_spiral()
spiral_o1o.merge_with(cap_o1o.ports[1], 0)
sketch.add_geometry(spiral_o1o)


path_i = [[0, 0], [40, 0]]
res_i = CPW(path_i, res_R)
res_i.merge_with(cap_i.ports[0], 0)
sketch.add_geometry(res_i)

In [416]:
#filter connection
Ch = cap.ports[1].position
Cv = cap1.ports[1].position
Ci = ind.ports[0].position
Xc = (Ch[0]+Cv[0]+Ci[0])/3
Yc = (Ch[1]+Cv[1]+Ci[1])/3

s_cpw = CPW.default_values['S']
w_cpw = CPW.default_values['W']

path_ff = [[Ch[0], Ch[1]], [Ch[0]+15, Ch[1]], [Xc-25, Yc-25], [Xc, Yc], [Xc+25, Yc+25], [Cv[0], Cv[1]-15], [Cv[0], Cv[1]]]
res_ff = CPW(path_ff, res_R)
sketch.add_geometry(res_ff)
sketch.circular_array(res_ff)

path_cf = [[Ci[0], Ci[1]], [Xc+5, Yc-5]]
res_cf = CPW(path_cf, res_R)
sketch.add_geometry(res_cf)
sketch.circular_array(res_cf)


cat5 = gdspy.Rectangle([Xc-s_cpw/2, Yc],
                       [Xc+s_cpw/2, Yc-20], layer = 0)
cat5.rotate(45/180*np.pi, [Xc, Yc])
cat4 = gdspy.Rectangle([Xc-s_cpw/2, Yc],
                       [Xc+s_cpw/2, Yc-20], layer = 0)
cat4.rotate(45/180*np.pi, [Xc, Yc])
cat4.rotate(90/180*np.pi)
cat3 = gdspy.Rectangle([Xc-s_cpw/2, Yc],
                       [Xc+s_cpw/2, Yc-20], layer = 0)
cat3.rotate(45/180*np.pi, [Xc, Yc])
cat3.rotate(180/180*np.pi)
cat2 = gdspy.Rectangle([Xc-s_cpw/2, Yc],
                       [Xc+s_cpw/2, Yc-20], layer = 0)
cat2.rotate(45/180*np.pi, [Xc, Yc])
cat2.rotate(-90/180*np.pi)


# sketch + cat5

Rectangle((1124.8774660940674, 1131.9485339059327), (1117.8063982822018, 1110.7353304703363), 0, 0)

In [417]:
res.merge_with(graphen_hole.ports[8], 0)
sketch.add_geometry(res)
CPW_c = CPW_C()
CPW_c.merge_with(res.ports[1], 0)
sketch.add_geometry(CPW_c)

res_r.merge_with(graphen_hole.ports[9], 0)
sketch.add_geometry(res_r)
CPW_c_r = CPW_C()
CPW_c_r.merge_with(res_r.ports[1], 0)
sketch.add_geometry(CPW_c_r)

sketch.assemble()
# sketch.show()




Assembling Empty Sketch:   0%|                                                                                                                                                                                                     | 0/40 [00:00<?, ?it/s]


Assembling Empty Sketch:  92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 37/40 [00:00<00:00, 336.38it/s]

In [418]:
# sketch.circular_array(res)
sketch.circular_array(spiral)
sketch.circular_array(spiral1)
sketch.circular_array(cap)
sketch.circular_array(cap1)
sketch.circular_array(cap_i)
sketch.circular_array(ind)
sketch.circular_array(marker)
# sketch.circular_array(CPW_c)

# Create some text
x0 = 5000/2-400
y0 = 5000/2-700
text = gdspy.Text("T", 90, (x0-145, y0+33))
# Create a rectangle extending the text's bounding box by 1
bb = np.array(text.get_bounding_box())
# rect = gdspy.Rectangle(x0+bb[0] - 10, y0+bb[1] + 10)
el = gdspy.Round((x0-120, y0+90), [100, 50], tolerance=1e-4)
el1 = gdspy.Round((x0-60, y0+0), [50, 20], tolerance=1e-4)
sketch + el1

# Subtract the text from the rectangle
inv = gdspy.boolean(el, text, "not")
sketch + inv
sketch.assemble()

sketch - cat5
sketch - cat4
sketch - cat3
sketch - cat2
# sketch.show()





Assembling Empty Sketch:   0%|                                                                                                                                                                                                     | 0/61 [00:00<?, ?it/s]



Assembling Empty Sketch:   7%|████████████▍                                                                                                                                                                                | 4/61 [00:00<00:01, 32.00it/s]



Assembling Empty Sketch:  13%|████████████████████████▊                                                                                                                                                                    | 8/61 [00:00<00:01, 32.00it/s]



Assembling Empty Sketch:  20%|████████████████████████████████████▉                                                                                                                                                       | 12/61 [00:00<0

'Empty Sketch', (0.0, 0.0), 0.0 deg, 7 ports

In [419]:
sketch.save_gds("C:/Users/mukha/Documents/Aalto/Graphen 10-11.22/Chips/2b____D2_line.gds")

In [420]:
# sketch.show()




Assembling Empty Sketch: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:20<00:00, 336.38it/s]



Assembling Empty Sketch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:19<00:00, 22.81it/s]